   #                                                   Compare files

This code is very strong. 
What it does, is that it compares 2 files. The user inputs 2 files, and it out puts with a series of values and does a series of checks to compare if document is real. 

# Import the files

In [ ]:
from skimage.measure import compare_ssim
import argparse
import imutils
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
%matplotlib inline
import statistics
from scipy import stats
import random
"""
DMatch.distance - Distance between descriptors. The lower, the better it is.
DMatch.trainIdx - Index of the descriptor in train descriptors

DMatch.queryIdx - Index of the descriptor in query descriptors
JUST NORMAL COUNT 1 TO EVERYTHING
DMatch.imgIdx - Index of the train image.
HAS TO BE 0 FOR BOTH M AND N

"""

# FILE LOCATIONS

In [ ]:
# read images
IMAGE1NAME = '/Users/kunal/Documents/VdartWorking/SampleCopies/FakeCopies/21JAN2020/Jalal-H1-copy.jpg'
IMAGE2NAME = '/Users/kunal/Documents/VdartWorking/SampleCopies/GenuineCopies/H1B-Anil GadePNG.PNG'
#REAL IMAGE
img1 = cv2.imread(IMAGE1NAME)
x11, y11, z11 = img1.shape
img1 = cv2.resize(img1, dsize=(698, 910), interpolation=cv2.INTER_CUBIC)
img2 = cv2.imread(IMAGE2NAME) 
x2, y2, z2 = img2.shape
img2 = cv2.resize(img2, dsize=(698, 910), interpolation=cv2.INTER_CUBIC)

# Image Size

if x11 == x2 and y11 == y2:
    print("ALL OKAY, Continuing")
elif x11 == 910 and y11 == 698:
    img2 = cv2.resize(img2, dsize=(698, 910), interpolation=cv2.INTER_CUBIC)
elif x2 == 910 and y2== 698:
    img1 = cv2.resize(img1, dsize=(698, 910), interpolation=cv2.INTER_CUBIC)

In [ ]:
print("Shape for image 1:")
print(x11, " ", y11, " ", z11)
print("Shape for image 2:")
print(x2, " ", y2, " ", z2)

# SSIM DIFFERENCE

In [ ]:
imageA = img1
imageB = img2


In [ ]:
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

print(imageA.shape)
print(imageB.shape)
bigH, bigW,other = imageA.shape

In [ ]:
(score, diff) = compare_ssim(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")
print("SSIM: {}".format(score))

In [ ]:
# threshold the difference image, followed by finding contours to
# obtain the regions of the two input images that differ
thresh = cv2.threshold(diff, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

areaDIFFERENCE = 0
# loop over the contours
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    cv2.rectangle(imageA, (x, y), (x + w, y + h), (0, 0, 255), 2)
    cv2.rectangle(imageB, (x, y), (x + w, y + h), (0, 0, 255), 2)
    areaDIFFERENCE += (w*h)

In [ ]:
perdiff =  ((areaDIFFERENCE/(bigW*bigH))*100)
print("The percent different is ", ((areaDIFFERENCE/(bigW*bigH))*100))

In [ ]:
#cv2.imshow("Original", imageA)
#cv2.imshow("Modified", imageB)
#cv2.imshow("Diff", diff)
#cv2.imshow("Thresh", thresh)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
# OVERLAY THE IMAGE

# Display All main points of connectors

In [ ]:
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

#keypoints
sift = cv2.xfeatures2d.SIFT_create()
keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
NUMBER_OF_KEYPOINST_IMG1 = len(keypoints_1)
NUMBER_OF_KEYPOINST_IMG2 = len(keypoints_2)

img_1 = cv2.drawKeypoints(gray1,keypoints_1,img1)
img_2 = cv2.drawKeypoints(gray2,keypoints_2,img2)


figure, ax = plt.subplots(1, 2, figsize=(16, 8))

ax[0].imshow(img1)
ax[1].imshow(img2)

# Display Both Images

In [ ]:
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

figure, ax = plt.subplots(1, 2, figsize=(16, 8))

ax[0].imshow(img1, cmap='gray')
ax[1].imshow(img2, cmap='gray')

# All lines compare

In [ ]:

sift = cv2.xfeatures2d.SIFT_create()

keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)
print(len(keypoints_1), len(keypoints_2))

bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

matches = bf.match(descriptors_1,descriptors_2)
#print(matches)
matches = sorted(matches, key = lambda x:x.distance)
totalMATCHES = (len(matches))
img3 = cv2.drawMatches(img1, keypoints_1, img2, keypoints_2, matches, img2, flags=6)#flags orginally 2 and must be even
im = Image.fromarray(img3)
im.save('/Users/kunal/Documents/VdartWorking/realFake/difference.png')
fig = plt.figure(figsize=(15,15))
plt.imshow(img3)
plt.show()


# Single line compare

In [ ]:

sift = cv2.xfeatures2d.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# BFMatcher with default params
bf = cv2.BFMatcher()
#print(des1, des2)
matches = bf.knnMatch(des1,des2, k=2)

"""
DMatch.distance - Distance between descriptors. The lower, the better it is.
DMatch.trainIdx - Index of the descriptor in train descriptors
DMatch.queryIdx - Index of the descriptor in query descriptors
DMatch.imgIdx - Index of the train image.
"""
"""
# Apply ratio test
good = []
#print(matches)
for m,n in matches:
    print(m.distance)
    print("             ", n.distance)
    if m.distance < 0.75*n.distance:
        good.append([m])
"""
test1 = matches[12:13]
print(test1)

img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches[12:13],None,flags =2)



plt.imshow(img3),plt.show()

In [ ]:
im = Image.fromarray(img3)
im

In [ ]:
cv2.drawMatchesKnn.__doc__

# Ratio threshold

In [ ]:
if img1 is None or img2 is None:
    print('Could not open or find the images!')
    exit(0)
#-- Step 1: Detect the keypoints using SURF Detector, compute the descriptors


minHessian = 400
detector = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
keypoints2, descriptors2 = detector.detectAndCompute(img2, None)
#-- Step 2: Matching descriptor vectors with a FLANN based matcher
# Since SURF is a floating-point descriptor NORM_L2 is used
matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
knn_matches = matcher.knnMatch(descriptors1, descriptors2, 2)
#-- Filter matches using the Lowe's ratio test
ratio_thresh = 0.7
good_matches = []
for m,n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)
print((len(good_matches)))
#-- Draw matches
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#-- Show detected matches
#cv2.imshow('Good Matches', img_matches)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
rationVALarray = []
for i in range(0,(int((1.0/0.05)+(1)))):
    rationVALarray.append(format((i*0.05),'.2f'))

In [ ]:
lenTOTAL = []
for i in rationVALarray:
    good_matches=[]
    ratio_thresh = i
    for m,n in knn_matches:
        if m.distance < float(ratio_thresh) * n.distance:
            good_matches.append(m)
    #print(i, " ", len(good_matches))
    lenTOTAL.append(len(good_matches))

In [ ]:
img1 = cv2.imread('/Users/kunal/Documents/VdartWorking/realFake/DocumentTest.PNG')  
img2 = cv2.imread('/Users/kunal/Documents/VdartWorking/realFake/DocumentPic.PNG') 

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

In [ ]:
fig = plt.figure(figsize=(13,13))
plt.plot(rationVALarray, lenTOTAL, color='orange')
plt.xlabel('Ratio Threshold')
plt.ylabel('Output')
plt.title('Compare Threshold Graph')
plt.show()

In [ ]:
rationVALarray = []
for i in range(0,(int((1.0/0.05)+(1)))):
    rationVALarray.append(format((i*0.05),'.2f'))
aList = []
for i in range(10):
    minHessian = 400
    detector = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
    keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
    keypoints2, descriptors2 = detector.detectAndCompute(img2, None)
    #-- Step 2: Matching descriptor vectors with a FLANN based matcher
    # Since SURF is a floating-point descriptor NORM_L2 is used
    matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
    knn_matches = matcher.knnMatch(descriptors1, descriptors2, 2)
    #-- Filter matches using the Lowe's ratio test 
    lenTOTAL = []
    for i in rationVALarray:
        good_matches=[]
        ratio_thresh = i
        for m,n in knn_matches:
            if m.distance < float(ratio_thresh) * n.distance:
                good_matches.append(m)
        #print(i, " ", len(good_matches))
        lenTOTAL.append(len(good_matches))
    #print(lenTOTAL)
    aList.append(lenTOTAL)

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(10,10))
colors = ['b', 'g', 'r', 'c','m' ,'y', 'k','w','g', 'r']
for i in range(len(aList)):
    print(aList[i])
    plt.plot(rationVALarray, aList[i], color=colors[i])
plt.xlabel('Ratio Threshold')
plt.ylabel('Output')
plt.title('Compare Threshold Graph')
plt.show()

In [ ]:
#print(aList[i])
#count = i+1
#for j in range(len(aList[i])):
    #print(aList[i][count])
#print(aList[i][i])
totalCompare =[]
for k in range(len(aList[0])):
    singlecompareArray=[]
    for i in range(len(aList)):
        singlecompareArray.append(aList[i][k])
    totalCompare.append(singlecompareArray)

In [ ]:
dataset= aList[0]
mean= np.mean(dataset)
median = np.median(dataset)
mode= stats.mode(dataset)

print("Mean: ", mean)
print("Median: ", median)
print("Mode: ", mode)

In [ ]:
perfectArray = []
for i in range(len(totalCompare)):
    perfectArray.append(statistics.median(totalCompare[i]))

In [ ]:
fig = plt.figure(figsize=(13,13))
plt.plot(rationVALarray, perfectArray, color='blue',linewidth=7.0)
plt.xlabel('Ratio Threshold')
plt.ylabel('Output')
plt.title('Compare Threshold Graph')
plt.show()

In [ ]:
"""

BASED ON THIS GRAPH YOU CAN SEE THAT THE AVERAGE VALUE THAT IS BEST FOR THE GRAPH WILL BE 0.64

    /-------\         --------      |       | 
    |       |         |             |       |
    |       |         |             |       |
    |       |         |-----\        \------/
    |       |         |     |               |
    |       |   /\    |     |               |
    \-------/   \/    |-----/               | 
"""

# FINAL RUN

In [ ]:

if img1 is None or img2 is None:
    print('Could not open or find the images!')
    exit(0)

minHessian = 400
detector = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
keypoints2, descriptors2 = detector.detectAndCompute(img2, None)
#-- Step 2: Matching descriptor vectors with a FLANN based matcher
# Since SURF is a floating-point descriptor NORM_L2 is used
matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
knn_matches = matcher.knnMatch(descriptors1, descriptors2, 2)
#-- Filter matches using the Lowe's ratio test
ratio_thresh = 0.64
good_matches = []
for m,n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)
print((len(good_matches)))
#-- Draw matches
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#-- Show detected matches
im = Image.fromarray(img3)
im.save('/Users/kunal/Documents/VdartWorking/realFake/FINALWORKINGIMAGEPT1.png')
fig = plt.figure(figsize=(15,15))
plt.imshow(img3)
plt.show()
NUMEBROFMATCHESRATIOTEST = len(good_matches)

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# BFMatcher with default params
bf = cv2.BFMatcher()
#print(des1, des2)
matches = bf.knnMatch(des1,des2, k=2)


#85
# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 87:
        good.append([m])
    if n.distance < 87:
        good.append([n])
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags =2)

im = Image.fromarray(img3)
im.save('/Users/kunal/Documents/VdartWorking/realFake/FINALWORKINGIMAGEPT2.png')
fig = plt.figure(figsize=(15,15))
plt.imshow(img3)
plt.show()
NUMEBROFMATCHESDISTANCETEST = len(good)

# OUTPUT

In [ ]:
print("**************************")
print(IMAGE1NAME)
print(IMAGE2NAME)
print("**************************")
print("IMAGE 1 is FAKE")
print("IMAGE 2 is REAL")
print("**************************")
print("Shape of image 1 is ")
print(x11, " ", y11, " ", z11)
print("Shape of image 2 is")
print(x2, " ", y2, " ", z2)
print("**************************")
print("The SSIM score is ")
print(score)
print("**************************")
print("The Percent Difference is")
print(perdiff)
print("**************************")
print("The number of key points in image 1 is")
print(NUMBER_OF_KEYPOINST_IMG1)
print("The number of key points in image 2 is")
print(NUMBER_OF_KEYPOINST_IMG2)
print("**************************")
print("The total number of matches is")
print(totalMATCHES)
print("**************************")
print("Number of matches for the distance test is", NUMEBROFMATCHESDISTANCETEST)
print("Number of matches for the ratio test is", NUMEBROFMATCHESRATIOTEST)

In [ ]:
trueCount = 0
falseCount = 0
if score > 0.285:
    trueCount += 1
#not using percent difference

## Output - FALSE